# Aufgabe 2: Data Exploration und Analyse

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math

In [ ]:
# Datensatz einlesen
dataset = pd.read_csv('./DatenAusgegeben1.0.csv', delimiter=';', encoding = 'cp852')
dataset

Import seaborn und pyplot

In [ ]:
columns = ['Grundstück in qm', 'Zustand', 'Gebaut',
           'Renoviert', 'Zustand Fassade', 'Kellerflche in qm',
           'Erster Stock in qm', 'Zweiter Stock in qm', 'Wohnflche in qm', 
           'Schlafzimmer', 'Küchen', 'Küchenqualitt', 'Rume', 'Garagenkapazitt',
           'Verkaufsmonat', 'Verkaufsjahr', 'Preis']

Zuerst müssen Ausreißer im Datensatz gefunden werden. Hierfür eignen sich Boxplots sehr gut. Dabei können numerische Attribute in eine Figur gezeichnet werden:

In [ ]:
## Durch Subplots mehrere Plots auf einer Figure
fig, axes = plt.subplots(4, 5, figsize = (10, 15))
i = 0
plt.tight_layout(pad = 2)
for column in columns:
  chosenax = axes[math.floor(i / 5)][i % 5]
  chosenax.set_title(column)
  sb.boxplot(data = dataset[column], ax = chosenax)
  i = i + 1
plt.show()

Bei dem obenstehenden Diagramm zeigt sich, dass sich in einigen Spalten durchaus starke Ausreißer finden.  
Hier ist als Beispiel die Grundstücksgröße, Wohnfläche und den Preis zu nennen. Diese Ausreißer sollten in der Data Preparation folglich bereinigt werden

## Untersuchung auf einen Zusammenhang zwischen Preis und Verkaufszeitpunkt
Um zu Untersuchen ob der Zeitpunkt einen Einfluss auf den Verkaufspreis besitzt, untersuchen wir zwei Boxplots die den Zusammenhang zwischen Verkaufszeitpunkt und Verkaufspreis darstellen sollen.

In [ ]:
sb.boxplot(data = dataset[['Verkaufsjahr', 'Preis']], x = 'Verkaufsjahr', y = 'Preis')
plt.show()
sb.boxplot(data = dataset[['Verkaufsmonat', 'Preis']], x = 'Verkaufsmonat', y = 'Preis')
plt.show()

### Erkenntnis
Aus diesen zwei Diagrammen lässt sich kein Zusammenhang zwischen Zeitpunkt und Preis erkennen.

## Untersuchung des Datensatzes auf vernachlässigbare Hauseigenschaften (Spalten)

Da nicht-aussagekräftige Spalten nicht in unser Modell einfließen sollten, wird folgend der Datensatz auf vernachlässigbare bzw. nichtssagende Eigenschaften untersucht.

In [ ]:
attribute_columns = ['Grundstücksform', 'Steigung', 'Zone', 'Lage',
'Typ', 'Zustand', 'Zustand Fassade', 'Heizung',
'Heizungsqualitt', 'Klimaanlage', 'Schlafzimmer', 'Küchen',
'Küchenqualitt', 'Rume', 'Garage Typ', 'Garagenkapazitt', 'Pool'] # Alle Spalten die Immobilieneigenschaften repräsentieren

data_no_nan = dataset[attribute_columns].fillna('Nix') # Füllt leere Zeilen (bspw. bei Pool und GaragenTyp) mit einem Standardwert.

column_amount = len(data_no_nan.columns)
fig, axes = plt.subplots(math.ceil(column_amount / 4), 4, figsize = (25, 15))

i = 0
plt.tight_layout(pad = 3)
for column in data_no_nan.columns:
  chosenax = axes[math.floor(i / 4)][i % 4]
  chosenax.set_title(column)
  sb.countplot(data = data_no_nan, x = column, ax = chosenax)
  i = i + 1
plt.show()


### Erkenntnis
Nicht aussagekräftige Spalten scheinen zu sein:
- Pool
- Küchen(anzahl)
- Klimaanlage
- Heizung

## Korrelation zwischen Preis und Immmobilieneigenschaften
Um die Korrelation visualisieren zu können wird folgend eine einspaltige Heatmap genutzt.  
Aus dieser soll sich herauslesen lassen können welche Attribute ausschlaggebend für einen höheren, oder auch einen niedrigeren Preis sind.

In [ ]:
plt.figure(figsize = (20, 10))
sb.heatmap(data = dataset.fillna(0).corr()['Preis'].to_frame(), annot = True, cmap = 'viridis')
plt.show()

### Erkenntnis
In einer Korrelation mit dem Preis stehende Eigenschaften sind:
- Grundstücksgröße
- Baujahr
- Renovierungsjahr
- 

Wichtig könnte sein:
- Gebaut ?
- Renoviert
- Kellerfläche
- Wohnfläche
- Küchenqualtiät
- Räume
- Garagenkapazität

In [ ]:
sb.boxenplot(data = dataset, x = 'Küchenqualitt', y = 'Preis')

In [ ]:
sb.boxenplot(data = dataset, x = 'Zustand', y = 'Preis')

In [ ]:
plt.figure(figsize = (15,8))
sb.boxplot(data = dataset, x = 'Preis', y = 'Bezirk')
plt.show()

Jeder Bezirk besitzt anderen Preis-Median und eine andere Reichweite der Hauspreise.
Bezirk beeinflusst folglich den Preis.

In [ ]:
sb.countplot(data = dataset, y = 'Heizung')
plt.show()

In [ ]:
sb.countplot(data = dataset[['Pool']].fillna('Kein Pool'), y = 'Pool')
plt.show()

In [ ]:
sb.countplot(data = dataset[['Klimaanlage']], y = 'Klimaanlage')
plt.show()

In [ ]:
sb.countplot(data = dataset[['Heizungsqualitt']], y = 'Heizungsqualitt')
plt.show()

In [ ]:
sb.countplot(data = dataset[['Grundstücksform']], y = 'Grundstücksform')
plt.show()

In [ ]:
sb.countplot(data = dataset[['Zone']], y = 'Zone')
plt.show()